# Importar librerias

Una vez realizado el analisis de las variables, se identifico que el objeto del contrato es la variable que contiene mayor información para poder determinar el sector del contrato. Por esta razón, se realiza el procesamiento de la variable a continuación:

In [1]:
import os
import sys

import pandas as pd
import numpy as np
import unidecode
import re
import spacy


# librerias propias
dir_proyecto, _ = os.path.split(os.getcwd())
sys.path.append(dir_proyecto)
nlp = spacy.load("es_core_news_sm")


In [2]:
from funciones import *

In [3]:
# Cargamos los datos

cont_df = leer_parquet(os.path.join(dir_proyecto,'INPUTS'))

#Eliminan los NAN
df = cont_df[~cont_df['objeto_del_contrato'].isna()].reset_index(drop=True)

In [4]:
df.head(5)

,nombre_entidad,nit_entidad,departamento,ciudad,localizaci_n,orden,sector,rama,entidad_centralizada,proceso_de_compra,...,recursos_de_credito,recursos_propios,ultima_actualizacion,codigo_entidad,codigo_proveedor,objeto_del_contrato,fecha_inicio_liquidacion,fecha_fin_liquidacion,fecha_de_inicio_de_ejecucion,fecha_de_fin_de_ejecucion
0,MUNICIPIO DE VILLA DE LEYVA,891801268,Boyacá,Villa De Leyva,"Colombia, Boyacá , Villa De Leyva",Territorial,Servicio Público,Ejecutivo,Descentralizada,CO1.BDOS.4729907,...,0,0,2023-10-11T00:00:00.000,703646026,710345323,ADQUISICIÓN DE MENAJE Y EQUIPOS PARA DOTACIÓN ...,None,None,NaN,NaN
1,ALCALDIA DISTRITO DE RIOHACHA,892115007,La Guajira,Riohacha,"Colombia, La Guajira , Riohacha",Territorial,No aplica/No pertenece,Ejecutivo,Descentralizada,CO1.BDOS.4827117,...,0,1503887467,2023-10-27T00:00:00.000,702424037,720445238,PRESTAR EL SERVICIO DE ADMINISTRACIÓN DE LA CA...,None,None,NaN,NaN
2,REGIONAL DE ASEGURAMIENTO EN SALUD No7,900407224,Meta,Villavicencio,"Colombia, Meta, Villavicencio",Nacional,defensa,Judicial,Descentralizada,CO1.BDOS.4731595,...,0,0,2023-10-11T00:00:00.000,702313123,702817453,PRESTACIÓN DE SERVICIOS DE SALUD DE BAJA MEDIA...,2024-05-01 13:00:00,2024-09-30 23:00:00,NaN,NaN
3,GOBERNACION DEL QUINDIO,890001639,Quindío,Armenia,"Colombia, Quindío, Armenia",Territorial,No aplica/No pertenece,Ejecutivo,Centralizada,CO1.BDOS.4720473,...,0,0,2023-10-11T00:00:00.000,702948183,706532686,PRESTAR SERVICIOS DE ALQUILER LOGÍSTICO DE SON...,2023-12-01 16:00:00,2026-07-31 16:00:00,NaN,NaN
4,AGENCIA DE DESARROLLO RURAL,900948958,Distrito Capital de Bogotá,No Definido,"Colombia, Bogotá, Bogotá",Nacional,agricultura,Ejecutivo,Descentralizada,CO1.BDOS.4823138,...,0,2380939329,2023-10-06T00:00:00.000,701551798,704242254,AUNAR ESFUERZOS TÉCNICOS ADMINISTRATIVOS Y FIN...,None,None,NaN,NaN


In [5]:
def limpiar_texto(string):
    
    #Estandarizar acentuaciones
    out = unidecode.unidecode(string)

    #Poner en minúsculas
    out = out.lower()
    
    #Eliminar caracteres que no sean del alfabeto básico latino [A-Za-z0-9_] espacios y saltos de línea
    out = re.sub(r"[^\w\s]|\n", ' ', out)
    
    #Eliminar uno o varios dígitos seguidos
    out = re.sub("\d+", "", out)
    
    #Eliminar todos los espacios seguidos luego de reemplazar el texto
    out = re.sub('\s+', ' ', out)

    #NLP object
    out = nlp(out)
    #Eliminar palabras de parada 'stopwords'
    out = [token.text for token in out if not token.is_stop]
    out = " ".join(out)

    #Lematizar
    lemmas =[token.lemma_ for token in nlp(out)]
    # Convertir la lista de lemmas nuevamente a texto
    out = " ".join(lemmas)

    #Remover artículos (el/la/lo/etc.) o palabras de dos letras")
    out = [token.text for token in nlp(out) if len(token) > 2]

    return out


In [23]:
df ["objeto_del_contrato"] = df ["objeto_del_contrato"].apply(limpiar_texto)

In [25]:
df["objeto_del_contrato"].head(10)

0    [aunar, esfuerzo, recurso, recuperacion, cober...
1    [adquisicion, mobiliario, unidad, administrati...
2    [suministrar, material, formacion, area, agrop...
3    [contratar, prestacion, servicio, actividadir,...
4    [atender, sitio, critico, red, terciario, carg...
5    [contratisto, obligar, suministrar, asistencia...
6    [construccion, pista, pumptrack, nino, constru...
7    [analisis, viabilidad, nutricional, inclusion,...
8    [brindar, apoyo, corporacion, cultura, turismo...
9    [mejoramiento, locativo, modalidad, monto, ago...
Name: objeto_del_contrato, dtype: object

In [8]:
data = df.copy()

data.to_csv("../Data/muestra_sincont_todas_varia.csv", index=False)
data.to_csv("../OUTPUT/muestra_sincont_todas_varia.csv", index=False)